<a href=https://experimental-design.github.io/bofire/>
  <img width="350" src="https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/logos/bofire-long.png" alt="BoFire Logo" />
</a>

## Getting started with BoFire

In our [docs](https://experimental-design.github.io/bofire/), you can find all different options for the [BoFire installation](https://experimental-design.github.io/bofire/install/). For basic BoFire Bayesian optimization features using [BoTorch](https://botorch.org/) which depends on
[PyTorch](https://pytorch.org/), you need to run

```
pip install bofire[optimization]
```

For a more complete introduction to BoFire, please look in our [docs](https://experimental-design.github.io/bofire/).

### Optimization problem

Let us consider a test function for single-objective optimization - the [Himmelblau's function](https://en.wikipedia.org/wiki/Himmelblau%27s_function). The Himmelblau's function is a multi-modal function with four identical local minima used to test the performance of optimization algorithms. The optimization domain of the Himmelblau's function is illustrated below together with the four minima marked red.

[comment]: <> (TODO: Exchange link below with https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/tutorials/himmelblau.png)

<div style="text-align: center;">
    <img src="../graphics/tutorials/himmelblau.png"
    alt="Himmelblau's function"
    width="300"/>
</div>


### Defining the optimization output

Let's consider the single continuous output variable *y* of the Himmelblau's function with the objective to minimize it. In BoFire's terminology, we create a `MinimizeObjective` object to define the optimization objective of a `Continuous Output` feature.

In [1]:
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MinimizeObjective


objective = MinimizeObjective()
output_feature = ContinuousOutput(key="y", objective=objective)

For more details on `Output` features and `Objective` objects, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining the optimization inputs

For the two continuous input variables of the Himmelblau's function *x1* and *x2*, we create two `ContinuousInput` features including boundaries following BoFire's terminology.

In [2]:
from bofire.data_models.features.api import ContinuousInput


input_feature_1 = ContinuousInput(key="x1", bounds=(-5, 5))
input_feature_2 = ContinuousInput(key="x2", bounds=(-5, 5))

For more details on `Input` features, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining the optimization domain

In BoFire's terminology, `Domain` objects fully describe the search space of the optimization problem. `Input` and `Output` features are optionally bound with `Constraint` objects to specify allowed relationships between the parameters. Here, we will run an unconstrained optimization. For more details, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


In [3]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs


domain = Domain(
    inputs=Inputs(features=[input_feature_1, input_feature_2]),
    outputs=Outputs(features=[output_feature]),
)

### Draw candidates and execute experiments

Let's define the Himmelblau's function to evaluate points in the domain space.

In [4]:
def himmelblau(x1, x2):
    return (x1**2 + x2 - 11) ** 2 + (x1 + x2**2 - 7) ** 2


To initialize an iterative Bayesian optimization loop, let's first randomly draw 10 samples from the domain. In BoFire's terminology, those suggested samples are called `Candidates`.

In [5]:
candidates = domain.inputs.sample(10, seed=13)

print(candidates)

         x1        x2
0  1.059211  1.374497
1 -4.176967 -1.589341
2 -3.784766  4.674178
3 -2.449681  1.128971
4  3.055689 -2.594837
5 -4.779897  3.157958
6 -0.040224 -2.291381
7  3.854749  3.860476
8 -0.896600  1.240756
9  0.536098 -0.041359


Let's execute the randomly drawn candidates using the `himmelblau` function to obtain `Experiments` in BoFire's terminology.

In [6]:
experimental_output = candidates.apply(
    lambda row: himmelblau(row["x1"], row["x2"]), axis=1
)

experiments = candidates.copy()
experiments["y"] = experimental_output

print(experiments)

         x1        x2           y
0  1.059211  1.374497   88.725854
1 -4.176967 -1.589341   98.436483
2 -3.784766  4.674178  186.371901
3 -2.449681  1.128971   81.809936
4  3.055689 -2.594837   25.904959
5 -4.779897  3.157958  228.427101
6 -0.040224 -2.291381  179.821177
7  3.854749  3.860476  197.842843
8 -0.896600  1.240756  120.611344
9  0.536098 -0.041359  157.407519


For more details on candidates and experiments, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining an optimization strategy

Let's specify the strategy how the Bayesian optimization campaign should be conducted. Here, we define a single-objective Bayesian optimization strategy and pass the optimization domain together with a acquisition function. Here, we use logarithmic expected improvement `qLogEI` as the acquisition function. In BoFire's terminology, we create a serializable data model `SoboStrategy` which we then map to our functional model.


In [7]:
import bofire.strategies.api as strategies
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.strategies.api import SoboStrategy


sobo_strategy_data_model = SoboStrategy(
    domain=domain, acquisition_function=qLogEI(), seed=19
)

sobo_strategy = strategies.map(sobo_strategy_data_model)

c:\Users\chrihaas\AppData\Local\miniforge3\envs\autolc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For more details on strategy data models and functional models, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Run the optimization loop

To run the optimization loop using BoFire's terminology, we first `tell` the strategy object about the experiments we have already executed.


In [8]:
sobo_strategy.tell(experiments=experiments)

Subsequently, we run the optimization loop with a budget of 30 iterations. In each iteration, we `ask` the strategy object for one new candidate (returned as a list with one item). Then, we execute the suggested candidate to obtain a new experiment. To complete one full iteration, we concatenate the new experiment to the existing experiments and `tell` the strategy about the updated experiments.

In [9]:
import pandas as pd


for _ in range(30):
    new_candidates = sobo_strategy.ask(candidate_count=1)

    new_experiments = new_candidates.copy()
    new_experiments["y"] = new_candidates.apply(
        lambda row: himmelblau(row["x1"], row["x2"]), axis=1
    )
    experiments = pd.concat([experiments, new_experiments], join="inner").reset_index(
        drop=True
    )

    sobo_strategy.tell(experiments=experiments)

c:\Users\chrihaas\AppData\Local\miniforge3\envs\autolc\Lib\site-packages\botorch\optim\fit.py:104: OptimizationWarning: `scipy_minimize` terminated with status OptimizationStatus.FAILURE, displaying original message from `scipy.optimize.minimize`: ABNORMAL_TERMINATION_IN_LNSRCH
  warn(


The optimization behavior of the strategy is shown in the animated figure below. The four minima are marked red, the experiments carried out are marked blue with blue lines connecting them. The contours are indicating the predicted mean of the current model of each iteration.

[comment]: <> (TODO: Update Figure with Dorofee and exchange link below with https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/tutorials/himmelblau_optimization.gif)

<div style="text-align: center;">
    <img src="../graphics/tutorials/himmelblau_optimization.gif"
    alt="Optimization of Himmelblau's function"
    width="300"/>
</div>

# APPENDIX: Himmelblau graphics

This section shows the code to produce the Himmelblau graphics in the getting_started.ipynb notebook and the readme section.

In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
from PIL import Image


plotly_config = {
    'toImageButtonOptions': {
        'format': 'png',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'scale': 4  # Multiply title/legend/axis/canvas sizes by this factor
    }
}


#### Explore Himmelblau Function

The function has 4 minima

In [11]:
x_1_vec = np.linspace(-6, 6, 100)
x_2_vec = np.linspace(-6, 6, 100)

x_1, x_2 = np.meshgrid(x_1_vec, x_2_vec)
results = himmelblau(x_1, x_2)

x_minima = [
        [3.0, 2.0],
        [-2.805118, 3.131312],
        [-3.779310, -3.283186],
        [3.584428, -1.848126],
    ]
y_minima = np.zeros(4)


#### Contour plot

In [12]:
fig = go.Figure(data=go.Contour(z=results, x=x_1_vec, y=x_2_vec, contours_coloring="heatmap", zmin=0, zmax=600))
fig.update_layout(
    title='Himmelblau Function',
    autosize=False,
    width=750,
    height=750,
    margin={"l": 65, "r": 50, "b": 65, "t": 90},
    scene={
        "xaxis_title": 'x_1',
        "yaxis_title": 'x_2',
        "zaxis_title": 'y',
    })


fig.update_xaxes(range=[-5, 5])
fig.update_yaxes(range=[-5, 5])

for i in range(4):
    fig.add_trace(go.Scatter(x=[x_minima[i][0]], y=[x_minima[i][1]],
                             mode='markers', showlegend=False,
                             marker={"size": 10, "color": 'red'}))
fig.show(config=plotly_config)

#### 3D plot

In [13]:
fig = go.Figure(data=go.Surface(z=results, x=x_1_vec, y=x_2_vec))
fig.update_layout(
    title='Himmelblau Function',
    autosize=False,
    width=750,
    height=750,
    margin={"l": 65, "r": 50, "b": 65, "t": 90},
    scene={
        "xaxis_title": 'x_1',
        "yaxis_title": 'x_2',
        "zaxis_title": 'y',
    })
for i in range(4):
    fig.add_trace(go.Scatter3d(x=[x_minima[i][0]], y=[x_minima[i][1]], z=[y_minima[i]],
                             mode='markers', showlegend=False,
                             marker={"size": 10, "color": 'red'}))
fig.show()

#### Optimization Outputs

In [14]:
objective = MinimizeObjective()
output_feature = ContinuousOutput(key="y", objective=objective)

#### Optimization Inputs


In [15]:
input_feature_1 = ContinuousInput(key="x1", bounds=(-6, 6))
input_feature_2 = ContinuousInput(key="x2", bounds=(-6, 6))

#### Optimization Domain

In [16]:
domain = Domain(
    inputs=Inputs(features=[input_feature_1, input_feature_2]),
    outputs=Outputs(features=[output_feature]),
)

#### Draw candidates and execute experiments

In [17]:
candidates = domain.inputs.sample(10, seed=19)

experimental_output = candidates.apply(
    lambda row: himmelblau(row["x1"], row["x2"]), axis=1
)

experiments = candidates.copy()
experiments["y"] = experimental_output


#### Defining an optimization strategy

In [18]:
sobo_strategy_data_model = SoboStrategy(
    domain=domain, acquisition_function=qLogEI(), seed=19
)

sobo_strategy = strategies.map(sobo_strategy_data_model)

#### Define function to plot model mean prediction

In [19]:
def plot_himmelblau_opt(samples, model_prediction, iteration):
    fig = go.Figure(data=go.Contour(z=model_prediction, x=x_1_vec, y=x_2_vec, contours_coloring="heatmap", colorbar={"title": 'y'}, zmin=0, zmax=600))
    fig.update_layout(
        title=f'Optimization iteration: {iteration}',
        autosize=False,
        width=400,
        height=350,
        margin={"l": 10, "r": 10, "b": 10, "t": 40},
        xaxis_title = 'x1',
        yaxis_title = 'x2',
        legend_title = 'y',
        yaxis={'range': [-6, 6]},
        xaxis={'range': [-6, 6]}
        )

    for i in range(4):
       fig.add_trace(go.Scatter(x=[x_minima[i][0]], y=[x_minima[i][1]],
                               mode='markers', showlegend=False,
                               marker={"size": 10, "color": 'red'}))
    fig.add_trace(go.Scatter(x=samples["x1"], y=samples["x2"], mode='lines', showlegend=False, line={"color": 'blue', "width": 1}))
    fig.add_trace(go.Scatter(
        x=samples["x1"],
        y=samples["x2"],
        mode='markers',
        showlegend=False,
        marker={
            "size": 5,
            "color": 'white',
            "symbol": 'circle-open'}))
    return fig

#### Run the optimization loop

In [20]:
sobo_strategy.tell(experiments=experiments)
prediction = sobo_strategy.predict(pd.DataFrame({"x1": x_1.flatten(), "x2": x_2.flatten()}))
pred = np.array(prediction["y_pred"]).reshape(100, 100)


for i in range(30):
    new_candidates = sobo_strategy.ask(candidate_count=1)

    new_experiments = new_candidates.copy()
    new_experiments["y"] = new_candidates.apply(
        lambda row: himmelblau(row["x1"], row["x2"]), axis=1
    )
    experiments = pd.concat([experiments, new_experiments], join="inner").reset_index(
        drop=True
    )

    sobo_strategy.tell(experiments=experiments)

    clear_output(wait=True)

    # Plot the optimization process
    prediction = sobo_strategy.predict(pd.DataFrame({"x1": x_1.flatten(), "x2": x_2.flatten()}))
    pred = np.array(prediction["y_pred"]).reshape(100, 100)
    fig = plot_himmelblau_opt(experiments, pred, i)
    fig.show(config=plotly_config)

    fig.write_image(f"himmelblau_{i}.png", engine="orca")

#### Generate Himmelblau GIF

In [21]:
images = []

for i in range(30):
    try:
        img = Image.open(f"himmelblau_{i}.png")
        images.append(img)
    except FileNotFoundError:
        print(f"File himmelblau_{i}.png not found. Skipping this file.")

if images:
    images[0].save("himmelblau_optimization.gif", save_all=True, append_images=images[1:], duration=600, loop=0)
    print("GIF successfully created as himmelblau_optimization.gif.")
else:
    print("No images found to create a GIF.")

GIF successfully created as himmelblau_optimization.gif.
